In [5]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np

%matplotlib widget

In [1]:
import cadquery as cq
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show, 
                                       replay, enable_replay, disable_replay, reset_replay)

from jupyter_cadquery import set_sidecar, set_defaults
show_object = show

Overwriting auto display for cadquery Workplane and Shape


In [11]:
set_sidecar("Gears")

## **Utility functions**

In [2]:
def sphere_to_cartesian(r, gamma, theta):
    '''Convert spherical coordinates to cartesian
       r - sphere radius
       gamma - polar angle
       theta - azimuth angle
       return - cartesian coordinates
    '''
    return (r * np.sin(gamma) * np.sin(theta),
            r * np.sin(gamma) * np.cos(theta),
            r * np.cos(gamma))

def s_arc(sr, c_gamma, c_theta, r_delta, start, end, n=32):
    '''Get arc points plotted on a sphere's surface
       sr - sphere radius
       c_gamma - polar angle of the circle's center
       c_theta - azimuth angle of the circle's center
       r_delta - angle between OC and OP, where O is sphere's center,
                 C is the circle's center, P is any point on the circle
       start - arc start angle
       end - arc end angle
       n - number of points
       return - circle points
    '''
    t = np.expand_dims(np.linspace(start, end, n), axis=1)
    a = sphere_to_cartesian(1.0, c_gamma + r_delta, c_theta)
    k = sphere_to_cartesian(1.0, c_gamma, c_theta)
    c = np.cos(t) * a + np.sin(t) * np.cross(k, a) + \
        np.dot(k, a) * (1.0 - np.cos(t)) * k
    c = c * sr
    return [dim.squeeze() for dim in np.hsplit(c, 3)]

def s_inv(gamma0, gamma):
    phi = np.arccos(np.tan(gamma0) / np.tan(gamma))    
    return np.arccos(np.cos(gamma) / np.cos(gamma0)) / np.sin(gamma0) - phi


def circle3d_by3points(a, b, c):
    u = b - a
    w = np.cross(c - a, u)
    u = u / np.linalg.norm(u)
    w = w / np.linalg.norm(w)
    v = np.cross(w, u)
    
    bx = np.dot(b - a, u)
    cx, cy = np.dot(c - a, u), np.dot(c - a, v)
    
    h = ((cx - bx / 2.0) ** 2 + cy ** 2 - (bx / 2.0) ** 2) / (2.0 * cy)
    cc = a + u * (bx / 2.0) + v * h
    r = np.linalg.norm(a - cc)

    return r, cc


def rotation_matrix(axis, alpha):
    ux, uy, uz = axis
    sina, cosa = np.sin(alpha), np.cos(alpha)
    r_mat = np.array((
        (cosa + (1.0 - cosa) * ux ** 2,
         ux * uy * (1.0 - cosa) - uz * sina,
         ux * uz * (1.0 - cosa) + uy * sina),
        (uy * ux * (1.0 - cosa) + uz * sina,
         cosa + (1.0 - cosa) * uy ** 2,
         uy * uz * (1.0 - cosa) - ux * sina),
        (uz * ux * (1.0 - cosa) - uy * sina,
         uz * uy * (1.0 - cosa) + ux * sina,
         cosa + (1.0 - cosa) * uz ** 2),
    ))

    return r_mat


def angle_between(o, a, b):
    p = a - o
    q = b - o
    return np.arccos(np.dot(p, q) / (np.linalg.norm(p) * np.linalg.norm(q)))


def project_to_xy_from_sphere_center(pts, sphere_r):
    gammas = np.arccos(np.dot(pts, np.array((0.0, 0.0, 1.0))) / \
                       np.linalg.norm(pts, axis=1))
    thetas = np.arctan2(pts[:, 0], pts[:, 1])
    radiuses = sphere_r /  np.cos(gammas)
    proj_pts = np.dstack(sphere_to_cartesian(radiuses,
                                             gammas, thetas)).squeeze()
    
    return proj_pts


## **OpenCascade utility functions**

In [50]:
from OCP.Precision import Precision
from OCP.TColgp import TColgp_HArray2OfPnt, TColgp_HArray1OfPnt
from OCP.GeomAPI import (GeomAPI_PointsToBSplineSurface,
                         GeomAPI_PointsToBSpline)
from OCP.BRepBuilderAPI import BRepBuilderAPI_MakeFace


def make_spline_approx(points, tol=1e-2, smoothing=False, minDeg=1, maxDeg=3):
    '''
    Approximate a surface through the provided points.
    '''
    points_ = TColgp_HArray2OfPnt(1, len(points), 1, len(points[0]))

    for i, vi in enumerate(points):
        for j, v in enumerate(vi):
            v = cq.Vector(*v)
            points_.SetValue(i + 1, j + 1, v.toPnt())

    if smoothing:
        spline_builder = GeomAPI_PointsToBSplineSurface(
            points_, *smoothing, DegMax=maxDeg, Tol3D=tol
        )
    else:
        spline_builder = GeomAPI_PointsToBSplineSurface(
            points_, DegMin=minDeg, DegMax=maxDeg, Tol3D=tol
        )

    if not spline_builder.IsDone():
        raise ValueError("B-spline approximation failed")

    spline_geom = spline_builder.Surface()

    return cq.Face(BRepBuilderAPI_MakeFace(spline_geom, Precision.Confusion_s()).Face())


def make_spline_approx_1d(points, tol=1e-3, smoothing=None, minDeg=1, maxDeg=6):
    '''
    Approximate a spline through the provided points.
    '''
    points_ = TColgp_HArray1OfPnt(1, len(points))
    
    for ix, v in enumerate(points):
        v = cq.Vector(*v)
        points_.SetValue(ix + 1, v.toPnt())

    if smoothing:
        spline_builder = GeomAPI_PointsToBSpline(
                            points_, *smoothing, DegMax=maxDeg, Tol3D=tol
                         )
    else:
        spline_builder = GeomAPI_PointsToBSpline(
                            points_, DegMin=minDeg, DegMax=maxDeg, Tol3D=tol
                         )

    if not spline_builder.IsDone():
        raise ValueError("B-spline approximation failed")

    spline_geom = spline_builder.Curve()

    return cq.Edge(BRepBuilderAPI_MakeEdge(spline_geom).Edge())


## **Bevel gear**

In [420]:
class BevelGear:    
    def __init__(self, module=1.0, teeth_number=13,
                 cone_angle=45.0, face_width=4.0, bore=6.0,
                 pressure_angle=20.0, helix_angle=0.0,
                 addendum_cf=1.0, dedendum_cf=1.25, curve_points=16):
        m = module
        self.z = z = teeth_number
        self.curve_points = curve_points
        self.face_width = face_width
        self.bore = bore
        ka = addendum_cf
        kd = dedendum_cf
        alpha = np.radians(pressure_angle)
        
        # pitch cone angle
        self.gamma_p = gamma_p = np.radians(cone_angle)

        # base/pitch circle radius
        rp = m * z / 2.0
        
        # great sphere radius, also corresponds to pitch cone flank length
        self.gs_r = gs_r = rp / np.sin(gamma_p) 
        
        # angles of base, face and root cones 
        self.gamma_b = gamma_b = np.arcsin(np.cos(alpha) * np.sin(gamma_p))
        self.gamma_f = gamma_f = gamma_p + np.arctan(ka * m / gs_r)
        self.gamma_r = gamma_r = gamma_p - np.arctan(kd * m / gs_r)

        phi_r = s_inv(gamma_b, gamma_p);
        self.mirrpoint = mirrpoint = np.pi / z + 2.0 * phi_r
                
        self.tau = tau = np.pi * 2.0 / z
        gamma_tr = max(gamma_b, gamma_r)

        # Tooth left flank curve points
        gamma = np.linspace(gamma_tr, gamma_f, curve_points)
        theta = s_inv(gamma_b, gamma)
        self.tooth_lflank = np.dstack(sphere_to_cartesian(gs_r,
                                                          gamma,
                                                          theta)).squeeze()
        # Tooth tip curve points
        theta_tip = np.linspace(theta[-1], mirrpoint - theta[-1], curve_points)
        self.tooth_tip = np.dstack(sphere_to_cartesian(gs_r,
                                                       np.full(curve_points,
                                                               gamma_f),
                                                       theta_tip)).squeeze()
        # Tooth right flank curve points
        self.tooth_rflank = np.dstack(
                                sphere_to_cartesian(gs_r,
                                                    gamma[::-1],
                                                    mirrpoint - \
                                                        theta[::-1])).squeeze()

        # Tooth root curve points
        if gamma_r < gamma_b:
            p1 = self.tooth_rflank[-1]
            p2 = np.array(sphere_to_cartesian(gs_r, gamma_b, theta[0] + tau))
            p3 = np.array(sphere_to_cartesian(gs_r, gamma_r,
                                              (tau + mirrpoint) / 2.0))
            rr, rcc = circle3d_by3points(p1, p2, p3)
            rcc_gamma = np.arccos(np.dot(p3, rcc) / \
                                  (np.linalg.norm(p3) * np.linalg.norm(rcc)))
            p1p3 = angle_between(rcc, p1, p3)
            a_start = (np.pi - p1p3 * 2.0) / 2.0
            a_end = -a_start + np.pi
            self.tooth_root = np.dstack(s_arc(gs_r, gamma_r + rcc_gamma,
                                              (tau + mirrpoint) / 2.0,
                                              rcc_gamma,
                                              np.pi / 2.0 + a_start,
                                              np.pi / 2.0 + a_end,
                                              curve_points)).squeeze()
        else:
            r_theta = np.linspace(mirrpoint - theta[0],
                                  theta[0] + tau, curve_points)
            self.tooth_root = np.dstack(sphere_to_cartesian(
                                                        gs_r,
                                                        np.full(curve_points,
                                                                gamma_tr),
                                                        r_theta)).squeeze()


    def plot(self, plt_axes, sphere=True, pitch_circle=False,
             base_circle=False, face_circle=False, root_circle=False,
             t_matrix=None):
        if t_matrix is None:
            t_matrix = np.identity(3)
        
        tooth_pts = np.vstack((self.tooth_lflank, self.tooth_tip,
                               self.tooth_rflank, self.tooth_root))
        gear_pts = np.concatenate([
                    tooth_pts @ rotation_matrix((0.0, 0.0, 1.0),
                                                self.tau * i)
                                   for i in range(self.z)]) @ t_matrix
        
        if sphere:
            u = np.linspace(0.0, 2.0 * np.pi, 24)
            v = np.linspace(0.0, np.pi, 24)
            gs_x = self.gs_r * np.outer(np.cos(u), np.sin(v))
            gs_y = self.gs_r * np.outer(np.sin(u), np.sin(v))
            gs_z = self.gs_r * np.outer(np.ones(np.size(u)), np.cos(v))
            ax.plot_surface(gs_x, gs_y, gs_z, color='#aaa', alpha=0.1)

        c_points = self.curve_points * 2
        t = np.linspace(0.0, np.pi * 2.0, c_points)
        circles = ((pitch_circle , self.gamma_p, 'blue'),
                   (base_circle, self.gamma_b, 'gray'),
                   (face_circle, self.gamma_f, 'violet'),
                   (root_circle, self.gamma_r, 'red'))
        
        for do_plot, gamma, color in circles:
            if do_plot:
                pts = np.dstack(sphere_to_cartesian(
                                    np.full(c_points, self.gs_r),
                                            gamma, t)).squeeze()
                pts = pts @ t_matrix
                x, y, z = (ax.squeeze() for ax in np.hsplit(pts, 3))
                ax.plot3D(x, y, z, linestyle='dashed',
                          linewidth=1.0, alpha=0.6, c=color)
            
        pts_x, pts_y, pts_z = (ax.squeeze() for ax in np.hsplit(gear_pts, 3))
        plt_axes.plot3D(pts_x, pts_y, pts_z, linewidth=1.0, color='black')


    def build(self):
        
        # build full cone gear faces
        t_faces = []
        
        bottom_pos = np.cos(self.gamma_r) * self.gs_r
        top_pos = np.cos(self.gamma_f) * (self.gs_r - self.face_width)
        
        for fpts in (self.tooth_lflank,
                     self.tooth_tip,
                     self.tooth_rflank,
                     self.tooth_root):
            edge_bottom = project_to_xy_from_sphere_center(fpts, bottom_pos)
            edge_bottom[:, 2] = 0.0
            
            edge_top = project_to_xy_from_sphere_center(fpts, top_pos)
            edge_top[:, 2] = bottom_pos - top_pos
            
            ppts = np.stack((edge_bottom, edge_top))
            face = make_spline_approx(ppts)            
            t_faces.append(face)
            
        faces = []
        for i in range(self.z):
            for tf in t_faces:
                faces.append(tf.rotate(cq.Vector(0, 0, 0),
                                       cq.Vector(0, 0, 1),
                                       np.degrees(self.tau * i)))
            
        shell = cq.Shell.makeShell(faces)
        body = cq.Solid.makeSolid(shell)
        
        bc_r = bottom_pos * np.tan(self.gamma_f)
        tc_r = top_pos * np.tan(self.gamma_f)
        
        b_face = cq.Workplane('XY').workplane(offset=-1.0).circle(bc_r * 1.1).extrude(1.0).val() 
        t_face = cq.Workplane('XY').workplane(offset=bottom_pos - top_pos).circle(tc_r * 1.1).extrude(1.0).val() 
        
        body = (cq.Workplane('XY').add(body).cut(b_face).cut(t_face))
        
        arc_x1 = -np.tan(self.gamma_r) * bottom_pos
        alpha = np.arctan2(bottom_pos, arc_x1) + np.pi + (self.gamma_f - self.gamma_r)
        arc_x2 = -np.cos(alpha) * self.gs_r
        arc_y2 = np.sin(alpha) * self.gs_r + bottom_pos
                
        bt_cut = (cq.Workplane('XZ')
                  .moveTo(arc_x1, 0.0)
                  .radiusArc((arc_x2, arc_y2), self.gs_r)
                  .lineTo(-bc_r, arc_y2)
                  .lineTo(-bc_r, 0.0)
                  .close()
                  .consolidateWires()
                  .revolve())
        
        body = (cq.Workplane('XY').add(body).cut(bt_cut))
        
        arc_x1 = tc_r + 0.01
        arc_y1 = bottom_pos - top_pos
        alpha = np.arctan2(-top_pos, arc_x1) - (self.gamma_f - self.gamma_r)
        arc_x2 = np.cos(alpha) * (self.gs_r - self.face_width) + 0.01
        arc_y2 = np.sin(alpha) * (self.gs_r - self.face_width) + bottom_pos
                
        tp_cut = (cq.Workplane('XZ')
                  .moveTo(0.0, (bottom_pos - top_pos) + 0.01)
                  .lineTo(tc_r, (bottom_pos - top_pos) + 0.01)
                  .radiusArc((arc_x2, arc_y2), (self.gs_r - self.face_width))
                  .lineTo(0.0, arc_y2)
                  .close()
                  .consolidateWires()
                  .revolve())
        
        body = (cq.Workplane('XY').add(body).cut(tp_cut))
        
        body = (cq.Workplane('XY').add(body).moveTo(0.0, 0.0).circle(self.bore / 2.0).cutThruAll())
        
        display = PartGroup(
            [
                Part(body, 'BevelGear', '#ccf', show_edges=True),
            ],
            "BevelGear"
        )
        
        show(display, axes=True, grid=True, ortho=True, axes0=True, transparent=True)

g = BevelGear(module=5.0, teeth_number=13, cone_angle=45.0, face_width=15.0, bore=10.0)

g.build()

Done, using side car 'Gears'


In [391]:
?cq.Shell.makeShell

Signature: cq.Shell.makeShell(listOfFaces: Iterable[cadquery.occ_impl.shapes.Face]) -> 'Shell'
Docstring: <no docstring>
File:      ~/anaconda3/envs/cq21-jl3/lib/python3.8/site-packages/cadquery/occ_impl/shapes.py
Type:      method


In [6]:
class BevelGearPair:
    
    def __init__(self, module, gear_teeth, pinion_teeth, face_width,
                 axis_angle=90.0, gear_bore=3.0, pinion_bore=3.0,
                 pressure_angle=20.0, helix_angle=0.0, addendum_cf=1.0,
                 dedendum_cf=1.25, curve_points=16):
        
        self.axis_angle = axis_angle = np.radians(axis_angle)
        
        # Cone Angle of the Gear
        delta_gear = np.arctan(np.sin(axis_angle) / \
                               (pinion_teeth / gear_teeth + \
                                np.cos(axis_angle)))
        
        # Cone Angle of the Pinion
        delta_pinion = np.arctan(np.sin(axis_angle) / \
                               (gear_teeth / pinion_teeth + \
                                np.cos(axis_angle)))
        
        self.gear = BevelGear(module, gear_teeth, np.degrees(delta_gear),
                              face_width, gear_bore, pressure_angle,
                              helix_angle, addendum_cf, dedendum_cf,
                              curve_points)
        
        self.pinion = BevelGear(module, pinion_teeth, np.degrees(delta_pinion),
                                face_width, gear_bore, pressure_angle,
                                helix_angle, addendum_cf, dedendum_cf,
                                curve_points)


    def plot(self, plt_axes, omega=0.0,
             pitch_circle=False, base_circle=False,
             face_circle=False, root_circle=False):        
        g_mat = rotation_matrix((0.0, 0.0, 1.0),
                                -self.gear.mirrpoint / 2.0 + np.pi \
                                + np.pi / self.gear.z + omega)
        
        p_omega = -omega * (self.gear.z / self.pinion.z)
        p_mat = rotation_matrix((0.0, 0.0, 1.0),
                                -self.pinion.mirrpoint / 2.0 + p_omega)
        p_mat = p_mat @ rotation_matrix((1.0, 0.0, 0.0), -self.axis_angle)

        self.gear.plot(ax, True, pitch_circle, base_circle, face_circle,
                       root_circle, g_mat)
        self.pinion.plot(ax, False, pitch_circle, base_circle, face_circle,
                         root_circle, p_mat)

gears = BevelGearPair(module=1.0, gear_teeth=16, pinion_teeth=12, face_width=20.0, axis_angle=120.0)

In [7]:
plt.close()

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

def plot_gears(frame):
    ax.clear()
    gears.plot(ax, omega=np.pi * 2.0 * (frame / 300.0), pitch_circle=True)

gears.plot(ax, pitch_circle=True)
# anim = FuncAnimation(fig, plot_gears, 300, interval=100)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [138]:
r = 2.0

t = np.linspace(0.0, np.pi, 20)
x1 = np.cos(t * 2.0) * r
y1 = np.sin(t * 2.0) * r
z1 = t * 3.0

x2 = np.cos(np.pi / 2.0 + t * 2.0) * r
y2 = np.sin(np.pi / 2.0 + t * 2.0) * r
z2 = t * 3.0


p1 = np.dstack((x1, y1, z1)).squeeze()
p2 = np.dstack((x2, y2, z2)).squeeze()

grid = np.stack((p1, p2))

f1 = makeSplineApprox(grid)
f2 = f1.rotate(cq.Vector(0, 0, 0), cq.Vector(0, 0, 1), 90.0)
f3 = f1.rotate(cq.Vector(0, 0, 0), cq.Vector(0, 0, 1), 180.0)
f4 = f1.rotate(cq.Vector(0, 0, 0), cq.Vector(0, 0, 1), 270.0)

bottom = (cq.Workplane('XY')
          .rect(np.sqrt(2 * r ** 2), np.sqrt(2 * r ** 2))
          .rotate(cq.Vector(0, 0, 0), cq.Vector(0, 0, 1), 45.0)).val()

top = bottom.translate(cq.Vector(0.0, 0.0, np.pi * 3.0))
top = cq.Face.makeFromWires(top)
bottom = cq.Face.makeFromWires(bottom)

sh = cq.Shell.makeShell([bottom, f1, f2, f3, f4, top])
print(sh.wrapped)
body = cq.Solid.makeSolid(sh)

body = (cq.Workplane('XZ')
        .add(body)
        .moveTo(0.0, np.pi * 3.0 / 2.0)
        .circle(0.8)
        .cutThruAll())

show(body)

Done, using side car 'Gears'


In [102]:
?cq.Solid.makeSolid

Signature: cq.Solid.makeSolid(shell: cadquery.occ_impl.shapes.Shell) -> 'Solid'
Docstring: <no docstring>
File:      ~/anaconda3/envs/cq21-jl3/lib/python3.8/site-packages/cadquery/occ_impl/shapes.py
Type:      method
